In [173]:
import pandas as pd
import numpy as np
import sys
import os
import itertools
from sys import stdin
from IPython.display import display
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)
import matplotlib.pyplot as plt
%matplotlib inline

In [174]:
dfloan = pd.read_csv('2012_to_2014_loans_data.csv')
dfinst = pd.read_csv('2012_to_2014_institutions_data.csv')

D:\Work\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (3,4,5,7,8,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [187]:
def hmda_init(dfloan, dfinst):
    dfl = dfloan.copy()
    dfi = dfinst.copy()

    dfl['Respondent_ID'] = dfloan['Respondent_ID'].map(lambda x:format(x,'010d') if type(x) is not str else x)

    dfl['Census_Tract_Number'] = dfloan['Census_Tract_Number'].map(lambda x:x if type(x) is not str else (float(x) if not 'NA' in x else 0))
    dfl['Tract_to_MSA_MD_Income_Pct'] = dfloan['Tract_to_MSA_MD_Income_Pct'].map(lambda x:x if type(x) is not str else (float(x) if not 'NA' in x else 0))

    dfl['County_Code'] = dfloan['County_Code'].map(lambda x:x if type(x) is not str else (int(x) if not 'NA' in x else 0))
    dfl['FFIEC_Median_Family_Income'] = dfloan['FFIEC_Median_Family_Income'].map(lambda x:x if type(x) is not str else (int(x) if not 'NA' in x else 0))
    dfl['MSA_MD'] = dfloan['MSA_MD'].map(lambda x:x if type(x) is not str else (int(x) if not 'NA' in x else 0))
    dfl['Number_of_Owner_Occupied_Units'] = dfloan['Number_of_Owner_Occupied_Units'].map(lambda x:x if type(x) is not str else (int(x) if not 'NA' in x else 0))
    dfl['Applicant_Income_000'] = dfloan['Number_of_Owner_Occupied_Units'].map(lambda x:x if type(x) is not str else (int(x) if not 'NA' in x else 0))

    dfl['idx'] = dfl['As_of_Year'].apply(str)+dfl['Respondent_ID'].apply(str)+dfl['Agency_Code'].apply(str)
    dfi['idx'] = dfi['As_of_Year'].apply(str)+dfi['Respondent_ID'].apply(str)+dfi['Agency_Code'].apply(str)
    dfl['Respondent_Name'] = dfl['idx'].map(dfi.set_index('idx')['Respondent_Name_TS'])
#     dfl['Respondent_Parent_Name'] = dfl['idx'].map(dfi.set_index('idx')['Parent_Name_TS'])
#     dfl['Respondent_combined'] = dfl['Respondent_Name'].apply(str) + ':' + dfl['Respondent_Parent_Name'].apply(str)

    return dfl.drop(['idx'],1)

In [188]:
df_merged = hmda_init(dfloan, dfinst)
df_merged.columns

Index(['Agency_Code', 'Applicant_Income_000', 'As_of_Year',
       'Census_Tract_Number', 'County_Code', 'FFIEC_Median_Family_Income',
       'Loan_Amount_000', 'MSA_MD', 'Number_of_Owner_Occupied_Units',
       'Respondent_ID', 'Sequence_Number', 'State_Code',
       'Tract_to_MSA_MD_Income_Pct', 'MSA_MD_Description',
       'Loan_Purpose_Description', 'Agency_Code_Description',
       'Lien_Status_Description', 'Loan_Type_Description', 'State',
       'County_Name', 'Conforming_Limit_000', 'Conventional_Status',
       'Conforming_Status', 'Conventional_Conforming_Flag', 'Respondent_Name'],
      dtype='object')

In [189]:
df_filtered = df_merged.where(lambda x:x>0).dropna().copy()

def remove_outliers(data, col, lower, upper):
    data = data[data[col] > lower]
    data = data[data[col] < upper]
    return data

col = 'Loan_Amount_000'
pivots = df_merged[col].quantile([0.01,0.99])
df_filtered = remove_outliers(df_filtered, col, pivots[0.01], pivots[0.99])

col = 'Applicant_Income_000'
pivots = df_merged[col].quantile([0.01,0.99])
df_filtered = remove_outliers(df_filtered, col, pivots[0.01], pivots[0.99])


In [178]:
df_ext = df_filtered.where(lambda x:x>0).dropna().copy()

df_ext['Tract_Median_Family_Income'] = df_ext.FFIEC_Median_Family_Income * df_ext.Tract_to_MSA_MD_Income_Pct / 100
df_ext['Income_to_Tract_MFI_Pct'] = df_ext.Applicant_Income_000 * 1000 * 100 / df_ext.Tract_Median_Family_Income
df_ext['Loan_to_Tract_MFI_Pct'] = df_ext.Loan_Amount_000 * 1000 * 100 / df_ext.Tract_Median_Family_Income
df_ext['Loan_to_Income_Pct'] = df_ext.Loan_Amount_000 * 100 / df_ext.Applicant_Income_000

def set_label(data, col, pivots):
    data[col+'_l'] = pd.qcut(data[col], pivots, labels=range(len(pivots)-1))
    return data

pivots = [0,0.05,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95,1]

df_ext = set_label(df_ext, 'Loan_Amount_000', pivots)
df_ext = set_label(df_ext, 'Applicant_Income_000', pivots)
df_ext = set_label(df_ext, 'Tract_Median_Family_Income', pivots)
df_ext = set_label(df_ext, 'Income_to_Tract_MFI_Pct', pivots)
df_ext = set_label(df_ext, 'Loan_to_Tract_MFI_Pct', pivots)
df_ext = set_label(df_ext, 'Loan_to_Income_Pct', pivots)

df_ext.columns

Index(['Agency_Code', 'Applicant_Income_000', 'As_of_Year',
       'Census_Tract_Number', 'County_Code', 'FFIEC_Median_Family_Income',
       'Loan_Amount_000', 'MSA_MD', 'Number_of_Owner_Occupied_Units',
       'Respondent_ID', 'Sequence_Number', 'State_Code',
       'Tract_to_MSA_MD_Income_Pct', 'MSA_MD_Description',
       'Loan_Purpose_Description', 'Agency_Code_Description',
       'Lien_Status_Description', 'Loan_Type_Description', 'State',
       'County_Name', 'Conforming_Limit_000', 'Conventional_Status',
       'Conforming_Status', 'Conventional_Conforming_Flag', 'Respondent_Name',
       'Respondent_combined', 'Tract_Median_Family_Income',
       'Income_to_Tract_MFI_Pct', 'Loan_to_Tract_MFI_Pct',
       'Loan_to_Income_Pct', 'Loan_Amount_000_l', 'Applicant_Income_000_l',
       'Tract_Median_Family_Income_l', 'Income_to_Tract_MFI_Pct_l',
       'Loan_to_Tract_MFI_Pct_l', 'Loan_to_Income_Pct_l'],
      dtype='object')

In [180]:
(df_ext[df_ext.As_of_Year==2013])[['Loan_Amount_000', 'Applicant_Income_000','Loan_to_Income_Pct','Loan_to_Tract_MFI_Pct','Income_to_Tract_MFI_Pct']].describe(percentiles=[.01,.05, 0.25, 0.5, 0.75, 0.95,0.99])

,Loan_Amount_000,Applicant_Income_000,Loan_to_Income_Pct,Loan_to_Tract_MFI_Pct,Income_to_Tract_MFI_Pct
count,417183.000000,417183.000000,417183.000000,417183.000000,417183.000000
mean,264.846643,1366.556164,23.258114,261.401205,1460.539833
std,147.019232,528.046591,18.761656,132.510469,739.606567
min,31.000000,275.000000,1.138790,12.956843,172.559110
1%,49.000000,358.000000,3.202536,54.015821,345.312982
5%,80.000000,556.000000,5.464481,93.889190,531.923219
25%,157.000000,973.000000,11.250000,176.222557,915.712541
50%,235.000000,1315.000000,18.145695,242.552192,1313.587140
75%,345.000000,1725.000000,29.077207,320.097310,1845.008764
95%,561.000000,2309.000000,58.573192,486.465748,2877.716938


In [182]:
print(len(df_merged))
print(len(df_filtered))
print(len(df_ext))

1321158
1177149
1177149


In [185]:
df_sampled = pd.concat([
    df_ext[df_ext.As_of_Year==2012].sample(200000),
    df_ext[df_ext.As_of_Year==2013].sample(200000),
    df_ext[df_ext.As_of_Year==2014].sample(200000)
])

In [186]:
df_sampled.describe()

,Agency_Code,Applicant_Income_000,As_of_Year,Census_Tract_Number,County_Code,FFIEC_Median_Family_Income,Loan_Amount_000,MSA_MD,Number_of_Owner_Occupied_Units,Sequence_Number,State_Code,Tract_to_MSA_MD_Income_Pct,Conforming_Limit_000,Tract_Median_Family_Income,Income_to_Tract_MFI_Pct,Loan_to_Tract_MFI_Pct,Loan_to_Income_Pct
count,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,6.000000e+05,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000
mean,6.842568,1368.644812,2013.000000,3892.949521,149.437692,89942.369167,271.175278,36073.477523,1368.644812,1.485741e+05,38.318618,116.950963,540.767898,105354.389153,1462.816266,267.506903,23.789328
std,2.445282,528.217568,0.816497,3264.210244,235.112643,17944.928803,148.796240,15053.267016,528.217568,3.139176e+05,15.120577,37.400731,84.046770,41292.874138,744.119034,135.360058,19.107864
min,1.000000,275.000000,2012.000000,1.000000,1.000000,45300.000000,31.000000,12580.000000,275.000000,1.000000e+00,10.000000,19.170000,417.000000,15308.370000,169.934641,13.323096,1.120752
25%,7.000000,973.000000,2012.000000,322.240000,17.000000,73900.000000,161.000000,15680.000000,973.000000,2.271000e+03,24.000000,90.830000,458.000000,74905.740000,911.413113,180.943075,11.552722
50%,7.000000,1318.000000,2013.000000,4161.000000,43.000000,85600.000000,242.000000,47260.000000,1318.000000,1.467900e+04,51.000000,113.410000,535.000000,98801.010000,1313.376358,247.814386,18.588874
75%,9.000000,1729.000000,2014.000000,7014.070000,145.000000,105900.000000,354.000000,47894.000000,1729.000000,1.252662e+05,51.000000,138.910000,625.000000,128340.210000,1862.324514,326.754255,29.717682
max,9.000000,2879.000000,2014.000000,9728.000000,840.000000,113400.000000,904.000000,49020.000000,2879.000000,3.076577e+06,54.000000,350.670000,625.000000,266735.320000,5681.376463,3279.865632,314.685315
